## Step 1: Import necessary library

In [56]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

#remove warnings
import warnings
warnings.filterwarnings("ignore")


## Step 2: Explore the available data

In [57]:
# open the data
'''normalize the nested json file and convert it to dataframe'''
with open('/Users/yihuiwang/Documents/BeCode/My_project/Revenue Forcasting/asset/accounton_data.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
df = pd.json_normalize(data)

In [ ]:
#check first five rows of data
df.head()

In [ ]:
#check how many rows and columns 
df.shape

In [ ]:
#check the missinf value for the whole dataset
df.isna().sum()

## Step 3: Exploratory Data Analysis and Data Cleaning

In [61]:
#drop the companies which has not any value in revenue for all 6 years
df_new= df.dropna(how='all',
                    subset=['revenue.2020', 'revenue.2019','revenue.2018','revenue.2017','revenue.2016','revenue.2015'])

In [ ]:
#check how many rows and columns 
df_new.shape

In [ ]:
#find type of legal_form, company_category, and province
for col in df_new:
    if len(df_new[col].unique())<100:
        print("####################"+col+"####################")
        print(df[col].unique())
        print("                            ")

In [ ]:
#check if there is duplicate in company name
sum(df_new.duplicated(subset=['company_name']))

## Step 4: Check the correlation of the financial data with revenue per year

In [ ]:
#get heatmap of financial data per year, with the revenue of year after
year_list = [2019,2018,2017,2016,2015]
for year in year_list:
    year = pd.DataFrame(df_new,columns=['ebit.'+str(year),'ebitda.'+str(year), 'profit_and_loss_after_taxes.'+str(year), 'total_assets.'+str(year),
'total_liabilities.'+str(year),'operating_profit_and_loss.'+str(year),'financial_profit_and_loss.'+str(year),
'staff_count.'+str(year),'net_added_value.'+str(year),'staff_costs.'+str(year),'revenue.'+str(year+1)])
    corrMatrix = year.corr()
    sns.heatmap(corrMatrix, annot=True)
    plt.show()

## Step 5: Select the Feature and clean the data

In [ ]:
#drop some of non_numeric columns, keep only nace_code,vat_number,company_category, province 
df_clean = df_new.drop(df_new.columns[[0,3,4,5,6]], axis=1)
df_clean

In [ ]:
#check the missinf value after drop
df_clean.isna().sum()

In [70]:
#as shown in the correlation table, ebit=operating_profit_and_loss, drop it and keep only ebit,also drop ebitda
#total_assets=total_liability, drop it and keep only total_assets
#drop also columns of profit_and_loss_after_taxes and financial_profit_and_loss for their poor correlation
year_list = [2020,2019,2018,2017,2016,2015]
for year in year_list:
    df_clean = df_clean.drop(columns=['ebitda.'+str(year),'total_liabilities.'+str(year),'operating_profit_and_loss.'+str(year),
    'profit_and_loss_after_taxes.'+str(year),'staff_count.'+str(year),'financial_profit_and_loss.'+str(year)])   

In [ ]:
#check the first five rows
df_clean.head()

## Step 6: Fill the missing value 

#### Fillna Strategy 1: Fill with the median value of previous years

In [72]:
#fill the nan value in year 2020 of previous five years (2015,2016,2017,2018,2019)
features=['ebit','total_assets','staff_costs','revenue','net_added_value']
for feature in features:
    df_clean[f"{feature}.2020"]=df_clean[f"{feature}.2020"].fillna(df_clean[[f"{feature}.2019",f"{feature}.2018",f"{feature}.2017",f"{feature}.2016",f"{feature}.2015"]].median(axis=1))
    

In [ ]:
#check the missinf value again after the filling
sum(df_clean["ebit.2020"].isnull())

In [74]:
#fill the nan value in year 2019 of previous four years (2015,2016,2017,2018)
features=['ebit','total_assets','staff_costs','revenue','net_added_value']
for feature in features:
    df_clean[f"{feature}.2019"]=df_clean[f"{feature}.2019"].fillna(df_clean[[f"{feature}.2018",f"{feature}.2017",f"{feature}.2016",f"{feature}.2015"]].median(axis=1))

In [ ]:
#check the missinf value again after the filling
sum(df_clean["ebit.2019"].isnull())

In [76]:
#fill the nan value in year 2018 of previous 3 years (2015,2016,2017)
features=['ebit','total_assets','staff_costs','revenue','net_added_value']
for feature in features:
    df_clean[f"{feature}.2018"]=df_clean[f"{feature}.2018"].fillna(df_clean[[f"{feature}.2017",f"{feature}.2016",f"{feature}.2015"]].median(axis=1))

In [77]:
#fill the nan value in year 2017 of previous 2 years (2015,2016)
features=['ebit','total_assets','staff_costs','revenue','net_added_value']
for feature in features:
    df_clean[f"{feature}.2017"]=df_clean[f"{feature}.2017"].fillna(df_clean[[f"{feature}.2016",f"{feature}.2015"]].median(axis=1))

In [78]:
#fill the nan value in year 2016 of previous 1 years (2015)
features=['ebit','total_assets','staff_costs','revenue','net_added_value']
for feature in features:
    df_clean[f"{feature}.2016"]=df_clean[f"{feature}.2016"].fillna(df_clean[[f"{feature}.2015"]].median(axis=1))

In [79]:
#fill the nan value in year 2015 of median value of all 6 years
features=['ebit','total_assets','staff_costs','revenue','net_added_value']
for feature in features:
    df_clean[f"{feature}.2015"]=df_clean[f"{feature}.2015"].fillna(df_clean[[f"{feature}.2020",f"{feature}.2019",f"{feature}.2018",f"{feature}.2017",f"{feature}.2016",f"{feature}.2015"]].median(axis=1))

In [ ]:
#check the missing value of whole dataset after the filling
df_clean.isna().sum()

#### Fillna Stategy 2: Fill in the missing value of median based on company_category, province, and nace_code and year

In [ ]:
#transform the dataset
df_melted = pd.melt(df_clean, id_vars=['vat_number','company_category','province','nace_code'])
df_melted

In [ ]:
#split the financial part with year
df_melted[['financial_data','year']]=df_melted.variable.str.split('.',expand=True)
df_melted

In [ ]:
df_melted.drop(columns=['variable'])

In [ ]:
#transform the dataset
df_transform=df_melted.pivot(index=['vat_number','company_category','province','nace_code','year'],columns=['financial_data'], values='value')
#rename the revenue to current_revenue
df_transform=df_transform.rename(columns={'revenue':'current_revenue'})
df_transform

In [ ]:
#get information of the whole dataset
df_transform.info()

In [ ]:
#add next_year_revenue column
df_transform[f"next_year_revenue"] = df_transform.groupby('vat_number')['current_revenue'].shift(-1)
df_transform.head(10)

In [87]:
#reset index
df_transform=df_transform.reset_index()

In [ ]:
#get first two digitals of nace_code, to know the companies' activities
df_transform['nace_code_section'] =df_transform['nace_code'].astype(str).str[:2]
#reorder the column
df_transform = df_transform[['vat_number','company_category','province','nace_code','nace_code_section',
                            'year','ebit','net_added_value','staff_costs','total_assets','current_revenue','next_year_revenue']]
df_transform


In [ ]:
#select datafram which next_year_revenue is not nan
df_transform=df_transform[df_transform['next_year_revenue'].notna()]
df_transform

In [ ]:
#get the median value based on company category province, nace_code and year
df_median=df_transform.groupby(['company_category', 'province','nace_code_section','year'])['ebit','net_added_value', 'staff_costs','total_assets','current_revenue'].median()
df_median

In [94]:
df_median.to_csv('/Users/yihuiwang/Documents/BeCode/My_project/Revenue Forcasting/asset/median.csv')

In [95]:
#fill the missing value with median value based on company category, province, nace section and year
df_transform[['ebit','net_added_value', 'staff_costs','total_assets','current_revenue']] = df_transform.groupby(['company_category', 'province','nace_code_section','year'])['ebit','net_added_value', 'staff_costs','total_assets','current_revenue'].transform(lambda x: x.fillna(x.median()))

In [ ]:
#check nan value after filling
df_transform.isna().sum()

## Step 7:Finalize the training Data

In [ ]:
#check how many 0 in staff_costs
df_transform.loc[(df_transform['staff_costs'] == 0)]

In [ ]:
#check the missing value
df_transform.isna().sum()

In [ ]:
#drop current_revenue feature
train_df = df_transform.drop(columns=['current_revenue'])
train_df

In [104]:
#drop the rows, if all the features are missing
train_df= train_df.dropna(how='all',
                    subset=['ebit','net_added_value','staff_costs','total_assets'])

In [ ]:
#check the missing value
train_df.isna().sum()

In [106]:
#drop the missing value in staff_costs
train_df=train_df.dropna(subset = ['staff_costs'])

In [ ]:
#check the missing value again after drop
train_df.isna().sum()

In [ ]:
#check where staff_cost is 0
train_df.loc[(train_df['staff_costs'] == 0)]

In [ ]:
#drop staff_costs, as there are too many 0 in that feature
#also drop the unnecessay column for train the model
train_df=train_df.drop(columns=['staff_costs','vat_number','company_category','province','nace_code','nace_code_section','year'])
train_df.head()

In [110]:
#save the train data to csv file
train_df.to_csv('/Users/yihuiwang/Documents/BeCode/My_project/Revenue Forcasting/asset/final_data_new.csv')